In [1]:
# project: p12
# submitter: rarora23
# partner: none

In [2]:
import requests
import os, json
import pandas as pd
from pandas import DataFrame, Series
import math
from bs4 import BeautifulSoup

In [3]:
def download(filename, url):
    # We do not download again if the file already exists
    if os.path.exists(filename):
        return (str(filename) + " already exists!")
    else:
        r = requests.get(url)
        r.raise_for_status()
        data = r.text 
        path = os.path.join(filename)
        f = open(path, "w", encoding="utf-8")
        f.write(data)
        f.close()
    return (str(filename) + " created!")

In [4]:
download("countries.json", "https://raw.githubusercontent.com/msyamkumar/cs220-f20-projects/master/p12/countries.json")

'countries.json already exists!'

In [5]:
countries = pd.read_json('countries.json')
countries.head()

,area,birth-rate,coastline,continent,country,death-rate,gdp-per-capita,infant-mortality,literacy,net-migration,phones,population,region
0,756950,15.23,0.85,South America,Chile,5.81,9900,8.80,"96,2",0.00,"213,0",16134219,LATIN AMER. & CARIB
1,665,17.80,24.21,Asia,Bahrain,4.14,16900,17.27,"89,1",1.05,"281,3",698585,NEAR EAST
2,22966,28.84,1.68,North America,Belize,5.72,4900,25.69,"94,1",0.00,"115,7",287730,LATIN AMER. & CARIB
3,582650,39.72,0.09,Africa,Kenya,14.02,1000,61.47,"85,1",-0.10,"8,1",34707817,SUB-SAHARAN AFRICA
4,11854,33.05,3.12,Australia,Marshall Islands,4.78,1600,29.45,"93,7",-6.04,"91,2",60422,OCEANIA


In [6]:
#q1 How many countries do we have in our dataset?
len(countries)

174

In [7]:
#q2 what is the total population across all the countries in our dataset?
countries['population'].sum()

6261901793

In [8]:
download("capitals.json","https://raw.githubusercontent.com/msyamkumar/cs220-f20-projects/master/p12/capitals.json")

'capitals.json already exists!'

In [9]:
capitals = pd.read_json('capitals.json')
capitals.head()

,country,capital,latitude,longitude
0,Afghanistan,Kabul,34.516667,69.183333
1,Albania,Tirana,41.316667,19.816667
2,Algeria,Algiers,36.750000,3.050000
3,Anguilla,The Valley,18.216667,-63.050000
4,Argentina,Buenos Aires,-34.583333,-58.666667


In [10]:
#q3 What are the capital names in capitals.json?
sorted(list(capitals["capital"]))

['Abu Dhabi',
 'Abuja',
 'Accra',
 'Addis Ababa',
 'Algiers',
 'Amman',
 'Amsterdam',
 'Ankara',
 'Antananarivo',
 'Apia',
 'Ashgabat',
 'Asmara',
 'Astana',
 'Asuncion',
 'Athens',
 'Baghdad',
 'Baku',
 'Bamako',
 'Bangkok',
 'Beijing',
 'Beirut',
 'Belmopan',
 'Berlin',
 'Bern',
 'Bishkek',
 'Bissau',
 'Bogota',
 'Brasilia',
 'Bridgetown',
 'Brussels',
 'Bucharest',
 'Budapest',
 'Buenos Aires',
 'Bujumbura',
 'Cairo',
 'Canberra',
 'Caracas',
 'Castries',
 'Chisinau',
 'Colombo',
 'Conakry',
 'Copenhagen',
 'Dakar',
 'Damascus',
 'Dar es Salaam',
 'Dhaka',
 'Djibouti',
 'Doha',
 'Dublin',
 'Dushanbe',
 'Freetown',
 'Gaborone',
 'George Town',
 'Georgetown',
 'Guatemala City',
 'Hagatna',
 'Hamilton',
 'Hanoi',
 'Harare',
 'Havana',
 'Helsinki',
 'Islamabad',
 'Jakarta',
 'Jamestown',
 'Jerusalem',
 'Kabul',
 'Kampala',
 'Kathmandu',
 'Khartoum',
 'Kigali',
 'Kingston',
 'Kingstown',
 'Kuala Lumpur',
 'Kuwait City',
 'Kyiv',
 'La Paz',
 'Libreville',
 'Lilongwe',
 'Lima',
 'Lisbon',


In [11]:
#q4 What is the capital of Italy?
next(iter(capitals.loc[capitals['country']=='Italy']['capital']))

'Rome'

In [12]:
#q5 Which country's capital is Brussels?
next(iter(capitals.loc[capitals['capital']=='Brussels']['country']))

'Belgium'

In [13]:
#q6 Which 7 countries have the southern-most capitals?
list(capitals.sort_values("latitude")['country'].head(7))

['New Zealand',
 'Australia',
 'Uruguay',
 'Argentina',
 'Chile',
 'Lesotho',
 'Swaziland']

In [14]:
#q7 Which 10 countries have the capitals closest to the North Pole?
list((capitals.sort_values("latitude", ascending=False)['country'].head(10)))

['Iceland',
 'Finland',
 'Norway',
 'Estonia',
 'Sweden',
 'Latvia',
 'Russia',
 'Denmark',
 'Lithuania',
 'Belarus']

In [15]:
#q8 What is the smallest land-locked country in Asia?
#smallest area zero coastline
min_area = countries[countries["continent"] == 'Asia'].sort_values("area")
min_coast = min_area[min_area["coastline"] == 0]["country"]
min_coast.iloc[0]

'Bhutan'

In [16]:
#q9 What is the largest coastal country in Asia?
countries[countries["continent"] == 'Asia'].sort_values("area",ascending = False)['country'].iloc[0]

'China'

In [17]:
#q10 What is coastal country with the least population in South America?
countries[countries["continent"] == 'South America'].sort_values("population")['country'].iloc[0]

'Suriname'

In [18]:
# source referring Haversine formula http://www.movable-type.co.uk/scripts/gis-faq-5.1.html
def distance(lat1,lon1,lat2,lon2):
    #step-1 to convert lat and long from degree to radiands
    lat1_rad = math.radians(lat1)
    lat2_rad = math.radians(lat2)
    lon1_rad = math.radians(lon1)
    lon2_rad = math.radians(lon2)
    
    #req condition for problem 13
    if lat1 == lat2:
        if lon1 == lon2:
            return None
    else:
        #use Haversine formula
        dlon = lon2_rad - lon1_rad
        dlat = lat2_rad - lat1_rad

        a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = 3956 * c
        return distance
    
#creating function to get latitude of given country
def get_lat(country):
    lat = capitals.loc[capitals['country'] == country]
    return next(iter(lat['latitude']))

#creating function to get longitude of given country
def get_lon(country):
    lon = capitals.loc[capitals['country']== country]
    return next(iter(lon['longitude']))

#function gets lat and lon of given countries and returns thier distance
def get_dist(country1,country2):
    lat1 = get_lat(country1)
    lon1 = get_lon(country1)
    lat2 = get_lat(country2)
    lon2 = get_lon(country2)
    return distance(lat1,lon1,lat2,lon2)
    

In [19]:
#q11 What is the distance between Camp Randall Stadium and the Wisconsin State Capital?
distance(43.070231, -89.411893,43.074645, -89.384113)

1.433899492072933

In [20]:
#q12 What is the distance between Germany and Norway?
get_dist("Germany","Norway")

520.8581822565818

In [21]:
#q13 What are the distances between Italy, United Kingdom and Germany?
countries_list = ['Italy', 'United Kingdom', 'Germany']

def dist_countries(country1, country_list):
    dist = {}
    for country in country_list:
        dist[country] = get_dist(country1, country)
    return dist

def distance_finder(req_countries):
    distance_dict = {}
    for country in req_countries:
        distance_dict[country] = dist_countries(country, req_countries)  
    return DataFrame(distance_dict)
distance_finder(countries_list)

,Italy,United Kingdom,Germany
Italy,NaN,888.292884,734.273770
United Kingdom,888.292884,NaN,576.471991
Germany,734.273770,576.471991,NaN


In [22]:
#q14 What is the distance between every pair of countries in the North American continent?
na_countries = []
df_na = countries.loc[countries['continent']=='North America']
for country in df_na['country']:
    na_countries.append(country)
distance_na_countries = distance_finder(na_countries)
distance_na_countries

,Belize,Jamaica,Bermuda,Saint Vincent and the Grenadines,Cuba,Saint Lucia,United States,Anguilla,Grenada,Dominican Republic,...,Guatemala,Panama,El Salvador,Mexico,Honduras,Aruba,Puerto Rico,Canada,Cayman Islands,Haiti
Belize,NaN,789.018414,1819.621687,1855.892129,580.312298,1858.259032,1653.241431,1691.193780,1838.286238,1242.169598,...,215.765123,843.680767,246.800133,695.785628,240.659212,1291.291337,1489.932051,2085.591134,504.276319,1083.020129
Jamaica,789.018414,NaN,1237.526654,1089.007795,503.397286,1083.910052,1441.944484,902.263021,1083.744550,453.608854,...,938.174764,650.084947,875.148492,1462.799009,741.563982,588.467289,701.235128,1894.046648,312.970577,295.158445
Bermuda,1819.621687,1237.526654,NaN,1341.314132,1243.923668,1284.728804,822.389478,977.198981,1410.205972,1005.541421,...,2025.950477,1865.916963,2004.094146,2298.396835,1890.141981,1404.733960,957.564805,1077.953449,1363.587084,1058.855675
Saint Vincent and the Grenadines,1855.892129,1089.007795,1341.314132,NaN,1545.842510,61.580891,2022.639028,371.521628,82.984426,684.148344,...,1965.339269,1273.646507,1878.754779,2545.035009,1745.124377,595.054996,491.403681,2385.082925,1401.877888,826.839946
Cuba,580.312298,503.397286,1243.923668,1545.842510,NaN,1530.121046,1133.133178,1290.666656,1553.370650,864.913522,...,792.703526,994.595466,789.610167,1108.616812,699.073644,1090.157633,1095.105790,1584.376888,270.765230,719.409966
Saint Lucia,1858.259032,1083.910052,1284.728804,61.580891,1530.121046,NaN,1975.318852,321.316659,143.779415,665.536830,...,1973.846475,1300.638928,1889.457798,2544.237348,1755.076587,615.610369,458.352467,2332.868266,1395.979292,813.473853
United States,1653.241431,1441.944484,822.389478,2022.639028,1133.133178,1975.318852,NaN,1654.202212,2075.376717,1472.538949,...,1866.198587,2071.555890,1891.361134,1883.030337,1820.863391,1869.849761,1553.301023,455.959281,1377.274306,1432.645171
Anguilla,1691.193780,902.263021,977.198981,371.521628,1290.666656,321.316659,1654.202212,NaN,434.495496,449.236722,...,1834.246930,1275.941217,1762.043200,2355.347815,1626.339238,608.940378,201.719030,2013.611444,1200.367589,608.614662
Grenada,1838.286238,1083.744550,1410.205972,82.984426,1553.370650,143.779415,2075.376717,434.495496,NaN,700.424260,...,1939.480320,1225.569170,1850.180746,2530.683480,1717.765657,559.736144,529.885445,2445.610880,1396.426627,834.542153
Dominican Republic,1242.169598,453.608854,1005.541421,684.148344,864.913522,665.536830,1472.538949,449.236722,700.424260,NaN,...,1389.306969,920.103185,1321.133901,1907.930275,1185.940969,410.914762,NaN,1890.303732,752.259451,159.393141


In [23]:
#q15 What is the most central country in the North American continent?
central_country = distance_na_countries.mean().sort_values().index
central_country[0]

'Jamaica'

In [24]:
#q16 What is the least central country in North America?
central_country[-1]

'Canada'

In [25]:
#q17 How close is each country in North America to its nearest neighbor?
near_col = {}
dist_col = {}

for country in list(distance_na_countries.idxmin().index):
    near_col[country] = distance_na_countries.idxmin()[country]
for country in list((distance_na_countries.min()).index):
    dist_col[country] = ((distance_na_countries.min())[country])

DataFrame({"nearest": near_col,
           "distance":dist_col})

,nearest,distance
Belize,Guatemala,215.765123
Jamaica,Haiti,295.158445
Bermuda,United States,822.389478
Saint Vincent and the Grenadines,Saint Lucia,61.580891
Cuba,Cayman Islands,270.765230
Saint Lucia,Saint Vincent and the Grenadines,61.580891
United States,Canada,455.959281
Anguilla,Puerto Rico,201.719030
Grenada,Saint Vincent and the Grenadines,82.984426
Dominican Republic,Haiti,159.393141


In [26]:
#q18 How far is each country in North America to its furthest neighbor?
furthest_col = {}
distance_col = {}

for country in list(distance_na_countries.idxmax().index):
    furthest_col[country] = distance_na_countries.idxmax()[country]
for country in list((distance_na_countries.max()).index):
    distance_col[country] = ((distance_na_countries.max())[country])

DataFrame({"furthest": furthest_col,
           "distance": distance_col})

,furthest,distance
Belize,Canada,2085.591134
Jamaica,Canada,1894.046648
Bermuda,Mexico,2298.396835
Saint Vincent and the Grenadines,Mexico,2545.035009
Cuba,Barbados,1641.236050
Saint Lucia,Mexico,2544.237348
United States,Grenada,2075.376717
Anguilla,Mexico,2355.347815
Grenada,Mexico,2530.683480
Dominican Republic,Mexico,1907.930275


In [27]:
#q19 For birth-rate and death-rate, what are various summary statistics (e.g., mean, max, standard deviation, etc.)?
countries.describe()[["birth-rate","death-rate"]]

,birth-rate,death-rate
count,174.000000,174.000000
mean,22.463851,9.625172
std,11.278992,5.187143
min,8.250000,2.410000
25%,12.597500,6.027500
50%,20.010000,8.230000
75%,29.860000,11.715000
max,50.730000,29.740000


In [28]:
#q20 BeautifulSoup
download('capitals.html', 'https://raw.githubusercontent.com/msyamkumar/cs220-f20-projects/master/p12/techslides-snapshot.html')
capital = open("capitals.html", "r")
capital = capital.read()
doc = BeautifulSoup(capital, "html.parser")
tables = doc.find_all("table")
tbl = tables[0]
trs = tbl.find_all("tr")

country_list = []
td_list = []
for tr in trs:
    tds = tr.find_all("td")
    td_list.append(tds)
    header = td_list[0]
    data = td_list[1:]

country_list = []
for rows in data:
    country_dict = {}
    for i in range(len(header)):
        if "Country Name" in header[i].get_text():
            country_dict["country"] = rows[i].get_text()
        elif "Capital Name" in header[i].get_text():
            country_dict["capital"] = rows[i].get_text()
        elif "Capital Latitude" in header[i].get_text():
            country_dict["latitude"] = float(rows[i].get_text())
        elif "Capital Longitude" in header[i].get_text():
            country_dict["longitude"] = float(rows[i].get_text())
        else:
            continue
    country_list.append(country_dict)
capital_new = []
country_og = sorted(list(countries["country"]))
for i in range(len(country_list)):
    if country_list[i]['country'] in country_og:
        capital_new.append(country_list[i])
    else:
        continue

with open('my_capitals.json', 'w') as f:
    json.dump(capital_new, f)  
file = open("my_capitals.json")
data = file.read()
file.close()
data


'[{"country": "Afghanistan", "capital": "Kabul", "latitude": 34.51666667, "longitude": 69.183333}, {"country": "Albania", "capital": "Tirana", "latitude": 41.31666667, "longitude": 19.816667}, {"country": "Algeria", "capital": "Algiers", "latitude": 36.75, "longitude": 3.05}, {"country": "Anguilla", "capital": "The Valley", "latitude": 18.21666667, "longitude": -63.05}, {"country": "Argentina", "capital": "Buenos Aires", "latitude": -34.58333333, "longitude": -58.666667}, {"country": "Armenia", "capital": "Yerevan", "latitude": 40.16666667, "longitude": 44.5}, {"country": "Aruba", "capital": "Oranjestad", "latitude": 12.51666667, "longitude": -70.033333}, {"country": "Australia", "capital": "Canberra", "latitude": -35.26666667, "longitude": 149.133333}, {"country": "Austria", "capital": "Vienna", "latitude": 48.2, "longitude": 16.366667}, {"country": "Azerbaijan", "capital": "Baku", "latitude": 40.38333333, "longitude": 49.866667}, {"country": "Bahrain", "capital": "Manama", "latitude"